# Customer Bronze

#### Este cuaderno es para la ingesta en la capa broze de la población de clientes. Como se trata  de una población estática no formara parte del pipeline ya que solo se ejecutará una vez.


In [1]:
%run customers_transactions_generation

StatementMeta(, a616826a-be8d-453d-bf8d-4db6a5f1281c, 8, Finished, Available, Finished)

In [2]:
# Libraries to import 
import random
import pandas as pd
import datetime
import time
import pyspark.sql.functions as F

StatementMeta(, a616826a-be8d-453d-bf8d-4db6a5f1281c, 9, Finished, Available, Finished)

In [3]:
# Ejecutamos el codigo principal
inicio = time.time()

# Creamos la base de clientes
print("Creando dataframe... \n")
population = BankPopulation(2000000)
list_population = population.create_customers()
list_population_kyc = population.kyc_identification(list_population)
list_population_pep = population.is_customer_pep(list_population_kyc)
list_population_region = population.customer_region(list_population_pep)
list_population_employed = population.is_employed(list_population_region)
list_population_naics = population.industry_naic_code(list_population_employed)
list_population_ns = population.name_screaning_hit(list_population_naics)
list_population_ams = population.adverse_media_screaning_hit(list_population_ns)
list_population_birth = population.date_of_birth_minor(list_population_ams)
print("Dataframe creado \n")
fin = time.time()

print(f"El proceso tardó {round(fin - inicio, 2)} segundos")
display(list_population_birth)

StatementMeta(, a616826a-be8d-453d-bf8d-4db6a5f1281c, 10, Finished, Available, Finished)

Creando dataframe... 

Dataframe creado 

El proceso tardó 12.34 segundos


SynapseWidget(Synapse.DataFrame, 432b3705-b4b3-4fe7-9e69-f803c49bf103)

In [4]:
# Creamos el spark dataframe para guardarlo como archivo en formato delta
print("Creando delta file y tabla raw")
base_bank_population = spark.createDataFrame(list_population_birth)
base_bank_population = base_bank_population.withColumn("date_of_birth", F.to_date(F.col("date_of_birth"), "dd/MM/yyyy"))
display(base_bank_population)
base_bank_population.printSchema()

# Guardamos el archico con formato delta en el lakehouse
base_bank_population.write.format("delta")\
                          .mode("overwrite")\
                          .save("Files/bronze/customers")


# Creamos la tabla customers_raw
base_bank_population.write.format("delta")\
                          .mode("overwrite")\
                          .saveAsTable("customers_raw")

print("Delta file y tabla raw finalizada")

StatementMeta(, a616826a-be8d-453d-bf8d-4db6a5f1281c, 11, Finished, Available, Finished)

Creando delta file y tabla raw


SynapseWidget(Synapse.DataFrame, 32bd2cbd-e8b3-47fa-97e5-b12c5adc7baf)

root
 |-- customer_id: string (nullable = true)
 |-- is_kyc_identified: long (nullable = true)
 |-- is_customer_pep: long (nullable = true)
 |-- region: string (nullable = true)
 |-- is_employed: long (nullable = true)
 |-- naic_code: string (nullable = true)
 |-- name_screaning_hit: long (nullable = true)
 |-- adverse_media_screaning_hit: long (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- is_minor: long (nullable = true)

Delta file y tabla raw finalizada
